# 自定义层

构造一个没有任何参数的自定义层

In [2]:
import torch
import torch.nn.functional as F
from torch import nn

# 自定义层与自定义网络没有本质区别，因为他们都是`Module`的子类

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到更复杂的模型中

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(1.8626e-09, grad_fn=<MeanBackward0>)

带参数的层

In [10]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        # 自定义权重和偏差参数，必须使用Parameter来定义，他会知道要有梯度
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight.data

tensor([[-1.3496,  1.0829,  1.1469],
        [-1.2265, -2.4403,  0.2678],
        [-2.0943,  0.2774, -0.0986],
        [ 1.0643, -0.4559,  0.8944],
        [ 0.6937, -0.3541,  2.3643]])

使用自定义层直接执行前向传播计算

In [11]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 3.0656],
        [0.0000, 0.0000, 4.3991]])

使用自定义层构建模型

In [12]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[6.8213],
        [6.5770]])